In [7]:
import pandas as pd
import pymysql
from sql_con.conn import get_con_db
import os 
import pdfplumber
import re
from tqdm import tqdm
import docx

In [46]:
pdf_dict = {}  
docx_dict = {}
for root, dirs, files in os.walk('./book_pdf'):  
    for file in tqdm(files):       
        if os.path.splitext(file)[1] == '.pdf':  # 想要保存的文件格式
            print('正在提取文件 %s '%file)
            pdf_full_name = os.path.join(root,file)
            pdf_book_text = []
            if pdf_full_name != './book_pdf/执业医师考试大纲.pdf':
                with pdfplumber.open(pdf_full_name) as pdf:
                    for page in pdf.pages:
                        pdf_book_text.append(page.extract_text())
            path_string = file
            pattern = re.compile(r'([^<>/\\\|:""\*\?]+)\.\w+$')
            book_name = pattern.findall(path_string)
            pdf_dict[book_name[0]] = ','.join(pdf_book_text)
            print('文件 %s 提取完成'% file)
            
        if os.path.splitext(file)[1] == '.docx': 
            print('正在提取文件 %s '%file)
            docx_full_name = os.path.join(root,file)
            docx_book_text = []
            docx_file = docx.Document(docx_full_name)
            for p in docx_file.paragraphs:
                docx_book_text.append(p.text)
            path_string = file
            pattern = re.compile(r'([^<>/\\\|:""\*\?]+)\.\w+$')
            book_name = pattern.findall(path_string)
            docx_dict[book_name[0]] = ','.join(docx_book_text)
            print('文件 %s 提取完成'% file)            
book_dict = dict(pdf_dict, **docx_dict)

con = get_con_db()
sql = '''
    select disease_name from dzj.tb_disease
'''
disease = pd.read_sql(sql,con)
disease.drop_duplicates(subset='disease_name',keep='first',inplace=True)
disease.reset_index(inplace=True,drop=True)
disease['book_name'] = ''

for i,row in disease.iterrows():
    disease_name = row['disease_name']
    for j in book_dict:
        book_text = book_dict[j]
        if disease_name in book_text:
            if disease.loc[disease['disease_name'] == disease_name,'book_name'].values != '':
                    disease.loc[disease['disease_name'] == disease_name,'book_name'] += ','+j
            else:
                disease.loc[disease['disease_name'] == disease_name,'book_name'] = j
        else:
            continue
# disease_in_book = disease.to_csv('./book_pdf/疾病对应书籍_1.csv')

  0%|                                                         | 0/12 [00:00<?, ?it/s]

正在提取文件 《妇产科学》人卫第9版教材--高清彩色.pdf 


  8%|████                                             | 1/12 [00:57<10:35, 57.73s/it]

文件 《妇产科学》人卫第9版教材--高清彩色.pdf 提取完成
正在提取文件 22.眼科学_第9版_全书签_可复制检索.pdf 


 17%|████████▏                                        | 2/12 [02:27<12:45, 76.60s/it]

...

 83%|████████████████████████████████████████        | 10/12 [07:42<01:02, 31.27s/it]

文件 《内科学》人卫第9版教材--高清彩色.docx 提取完成
正在提取文件 30.口腔科学 第9版可复制.docx 


100%|████████████████████████████████████████████████| 12/12 [07:42<00:00, 38.54s/it]


文件 30.口腔科学 第9版可复制.docx 提取完成


100%|███████████████████████████████████████████████| 1/1 [00:00<00:00, 20068.44it/s]


In [47]:
disease['book_name'].value_counts()

                                                                                                35545
《外科学》人卫第9版教材--高清彩色                                                                                666
《内科学》人卫第9版教材--高清彩色                                                                                562
神经病学 第8版                                                                                          437
22.眼科学_第9版_全书签_可复制检索                                                                              392
                                                                                                ...  
22.眼科学_第9版_全书签_可复制检索,23.传染病学_第9版_全书签_可复制检索,神经病学 第8版,《内科学》人卫第9版教材--高清彩色,30.口腔科学 第9版可复制               1
23.传染病学_第9版_全书签_可复制检索,神经病学 第8版,《儿科学》人卫第9版教材--高清彩色,《外科学》人卫第9版教材--高清彩色,30.口腔科学 第9版可复制                 1
22.眼科学_第9版_全书签_可复制检索,皮肤性病学 第9版_张学军，郑捷主编2018年（彩图）可检索,《儿科学》人卫第9版教材--高清彩色                              1
《妇产科学》人卫第9版教材--高清彩色,22.眼科学_第9版_全书签_可复制检索,神经病学 第8版,《儿科学》人卫第9版教材--高清彩色              